In [1]:
import pandas as pd
import numpy as np
import re
from string import ascii_letters, digits
from utils import cleaning_dicts
import os

In [2]:
#print(cleaning_dicts.clean_team_pff)
print(cleaning_dicts.clean_team_pff_full)

{'arizona-cardinals': 'ari', 'atlanta-falcons': 'atl', 'baltimore-ravens': 'bal', 'buffalo-bills': 'buf', 'carolina-panthers': 'car', 'chicago-bears': 'chi', 'cincinnati-bengals': 'cin', 'cleveland-browns': 'cle', 'dallas-cowboys': 'dal', 'denver-broncos': 'den', 'detroit-lions': 'det', 'green-bay-packers': 'gb', 'houston-texans': 'hou', 'indianapolis-colts': 'ind', 'jacksonville-jaguars': 'jax', 'kansas-city-chiefs': 'kc', 'los-angeles-chargers': 'lac', 'los-angeles-rams': 'lar', 'miami-dolphins': 'mia', 'minnesota-vikings': 'min', 'new-england-patriots': 'ne', 'new-orleans-saints': 'no', 'new-york-giants': 'nyg', 'new-york-jets': 'nyj', 'oakland-raiders': 'lv', 'philadelphia-eagles': 'phi', 'pittsburgh-steelers': 'pit', 'san-francisco-49ers': 'sf', 'seattle-seahawks': 'sea', 'tampa-bay-buccaneers': 'tb', 'tennessee-titans': 'ten', 'washington-redskins': 'was', 'washington-football-team': 'was', 'las-vegas-raiders': 'lv', 'washington-commanders': 'was', 'washingtoncommanders': 'was', 

In [3]:
cur_dir = os.path.abspath(os.getcwd())
cur_week_folder = 'week_6'
cur_week = '6'
cur_week_str = '6'
cur_week_int = 6
cur_year_int = 2022

In [4]:
print(cur_dir)

/home/tomb/nfl_models


In [5]:
####################################################################################
                ###   Read-in and clean all passing datasets ###
####################################################################################


passing_depth = pd.read_csv('./scripts/nfl_all/passing_depth_2022.csv')
passing_allowed_pressure = pd.read_csv('./scripts/nfl_all/passing_allowed_pressure_2022.csv')
passing_pressure = pd.read_csv('./scripts/nfl_all/passing_pressure_2022.csv')
passing_concept = pd.read_csv('./scripts/nfl_all/passing_concept_2022.csv')
time_in_pocket = pd.read_csv('./scripts/nfl_all/time_in_pocket_2022.csv')
passing_summ_conc = pd.read_csv('./scripts/nfl_all/passing_summ_conc_2022.csv')



def drop_non_qbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df=df[df['position'] == 'QB']
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(cleaning_dicts.pos_dict).fillna(df['position'])

    
#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df
    
passing_depth_new = drop_non_qbs(passing_depth)
passing_allowed_pressure_new = drop_non_qbs(passing_allowed_pressure)
passing_pressure_new = drop_non_qbs(passing_pressure)
passing_concept_new = drop_non_qbs(passing_concept)
time_in_pocket_new = drop_non_qbs(time_in_pocket)
passing_summ_conc_new = drop_non_qbs(passing_summ_conc)


passing_depth_new = passing_depth_new[passing_depth_new.columns.drop(list(passing_depth_new.filter(regex='left|right|center')))]

####################################################################################
				###   Read-in and clean all receiving datasets ### scripts/nfl_all
####################################################################################
                                 
rec_summ_conc = pd.read_csv('./scripts/nfl_all/rec_summ_conc_2022.csv')
receiving_concept = pd.read_csv('./scripts/nfl_all/receiving_concept_2022.csv')
receiving_depth = pd.read_csv('./scripts/nfl_all/receiving_depth_2022.csv')
receiving_scheme = pd.read_csv('./scripts/nfl_all/receiving_scheme_2022.csv')
                                 
def drop_non_recs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|TE|HB|FB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(cleaning_dicts.pos_dict).fillna(df['position'])

    
#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df

rec_summ_conc = drop_non_recs(rec_summ_conc)
receiving_concept = drop_non_recs(receiving_concept)
receiving_depth = drop_non_recs(receiving_depth)
receiving_scheme = drop_non_recs(receiving_scheme)


####################################################################################
				###   Read-in and clean all rushing datasets ###
####################################################################################

rush_summ_conc = pd.read_csv('./scripts/nfl_all/rush_summ_conc_2022.csv')                                 


def drop_non_rbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|HB|FB|QB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(cleaning_dicts.pos_dict).fillna(df['position'])

    
#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df

rush_summ_conc = drop_non_rbs(rush_summ_conc)


####################################################################################
				###   Read-in and clean all blocking datasets ###
####################################################################################

                               
block_summ_conc = pd.read_csv('./scripts/nfl_all/block_summ_conc_2022.csv')
offense_pass_blocking = pd.read_csv('./scripts/nfl_all/offense_pass_blocking_2022.csv')
offense_run_blocking = pd.read_csv('./scripts/nfl_all/offense_run_blocking_2022.csv')                                 


def drop_non_ols(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df = df[df['position'].notna()]
    df= df[df.position.str.match('T|C|G|TE')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(cleaning_dicts.pos_dict).fillna(df['position'])

    
#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df


block_summ_conc	= drop_non_ols(block_summ_conc)
offense_pass_blocking = drop_non_ols(offense_pass_blocking)
offense_run_blocking = drop_non_ols(offense_run_blocking)



####################################################################################
				###   Read-in and clean all defensive datasets ###
####################################################################################

                                
def_summ_conc = pd.read_csv('./scripts/nfl_all/def_summ_conc_2022.csv')
pass_rush_summary = pd.read_csv('./scripts/nfl_all/pass_rush_summary_2022.csv')
run_defense_summary = pd.read_csv('./scripts/nfl_all/run_defense_summary_2022.csv')
defense_coverage_scheme = pd.read_csv('./scripts/nfl_all/defense_coverage_scheme_2022.csv')
defense_coverage_summary = pd.read_csv('./scripts/nfl_all/defense_coverage_summary_2022.csv')
slot_coverage = pd.read_csv('./scripts/nfl_all/slot_coverage_2022.csv')

                              
def drop_non_def(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(cleaning_dicts.pos_dict).fillna(df['position'])


#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df

def_summ_conc = drop_non_def(def_summ_conc)
pass_rush_summary = drop_non_def(pass_rush_summary)
run_defense_summary = drop_non_def(run_defense_summary)
defense_coverage_scheme = drop_non_def(defense_coverage_scheme)
defense_coverage_summary = drop_non_def(defense_coverage_summary)
slot_coverage = drop_non_def(slot_coverage)

def_summ_conc=def_summ_conc[def_summ_conc['position'].isin(["ed","lb","di","s","cb"])]
pass_rush_summary=pass_rush_summary[pass_rush_summary['position'].isin(["ed","lb","di","s"])]
run_defense_summary=run_defense_summary[run_defense_summary['position'].isin(["ed","lb","di","s","cb"])]
defense_coverage_scheme=defense_coverage_scheme[defense_coverage_scheme['position'].isin(["lb","s","cb"])]
defense_coverage_summary=defense_coverage_summary[defense_coverage_summary['position'].isin(["lb","s","cb"])]
slot_coverage=slot_coverage[slot_coverage['position'].isin(["lb","s","cb"])]

####################################################################################
				###   Read-in and clean all special teams datasets ###
####################################################################################	


st_kickers = pd.read_csv('./scripts/nfl_all/st_kickers_2022.csv')
st_punters = pd.read_csv('./scripts/nfl_all/st_punters_2022.csv')                                 
                                 
                               
def clean_spec(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(cleaning_dicts.pos_dict).fillna(df['position'])


#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df

st_kickers =clean_spec(st_kickers)
st_punters = clean_spec(st_punters)



####################################################################################
####################################################################################
####################################################################################

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:62: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:98: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:175: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:219: FutureWarni

In [6]:
def create_prefix(prefix=None, df=None):
    id = df[['p_id','player_team_id','unique_team_id','team_id_impute','player','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','unique_team_id','player','team_id_impute','numeric_id','position','team_name','unique_team_id','numeric_id','position','team_name','year','week'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

def id_prefix(prefix=None, df=None):
    id = df[['p_id','player','player_team_id','unique_team_id','team_id_impute','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','unique_team_id','team_id_impute','numeric_id','position','team_name','year','week'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

passing_summ_conc = id_prefix(prefix="pass_summary_", df=passing_summ_conc)
rush_summ_conc = id_prefix(prefix="rush_summary_", df=rush_summ_conc)
rec_summ_conc = id_prefix(prefix="rec_summary_", df=rec_summ_conc)
block_summ_conc = id_prefix(prefix="block_summary_", df=block_summ_conc)
def_summ_conc = id_prefix(prefix="def_summary_", df=def_summ_conc)
st_kickers = id_prefix(prefix="kicking_", df=st_kickers)
st_punters = id_prefix(prefix="punting_", df=st_punters)


passing_depth = create_prefix(prefix="pass_depth_", df=passing_depth)
passing_allowed_pressure = create_prefix(prefix="pressure_source_", df=passing_allowed_pressure)
passing_pressure = create_prefix(prefix="pass_under_pressure_", df=passing_pressure)
passing_concept = create_prefix(prefix="pass_concept_", df=passing_concept)
time_in_pocket = create_prefix(prefix="pass_time_", df=time_in_pocket)


receiving_concept = create_prefix(prefix="rec_concept_", df=receiving_concept)
receiving_depth = create_prefix(prefix="rec_depth_", df=receiving_depth)
receiving_scheme = create_prefix(prefix="rec_scheme_", df=receiving_scheme)

offense_pass_blocking = create_prefix(prefix="pass_block_", df=offense_pass_blocking)
offense_run_blocking = create_prefix(prefix="run_block_", df=offense_run_blocking)


pass_rush_summary = create_prefix(prefix="pass_rush_", df=pass_rush_summary)
run_defense_summary = create_prefix(prefix="run_defense_", df=run_defense_summary)
defense_coverage_scheme = create_prefix(prefix="def_coverage_scheme_", df=defense_coverage_scheme)
defense_coverage_summary = create_prefix(prefix="def_coverage_summary_", df=defense_coverage_summary)
slot_coverage = create_prefix(prefix="def_slot_coverage_", df=slot_coverage)


KeyError: "['p_id', 'player_team_id', 'unique_team_id', 'team_id_impute', 'numeric_id'] not in index"

In [ ]:

passing_depth = passing_depth[passing_depth['week'] >= cur_week_int-2]
passing_allowed_pressure = passing_allowed_pressure[passing_allowed_pressure['week'] >= cur_week_int-2]
passing_pressure = passing_pressure[passing_pressure['week'] >= cur_week_int-2]
passing_concept = passing_concept[passing_concept['week'] >= cur_week_int-2]
time_in_pocket = time_in_pocket[time_in_pocket['week'] >= cur_week_int-2]
passing_summ_conc = passing_summ_conc[passing_summ_conc['week'] >= cur_week_int-2]


rec_summ_conc = rec_summ_conc[rec_summ_conc['week'] >= cur_week_int-2]
receiving_concept =receiving_concept[receiving_concept['week'] >= cur_week_int-2]
receiving_depth = receiving_depth[receiving_depth['week'] >= cur_week_int-2]
receiving_scheme = receiving_scheme[receiving_scheme['week'] >= cur_week_int-2]

rush_summ_conc =rush_summ_conc[rush_summ_conc['week'] >= cur_week_int-2]

block_summ_conc = block_summ_conc[block_summ_conc['week'] >= cur_week_int-2]
offense_pass_blocking = offense_pass_blocking[offense_pass_blocking['week'] >= cur_week_int-2]
offense_run_blocking = offense_run_blocking[offense_run_blocking['week'] >= cur_week_int-2]

def_summ_conc = def_summ_conc[def_summ_conc['week'] >= cur_week_int-2]
pass_rush_summary = pass_rush_summary[pass_rush_summary['week'] >= cur_week_int-2]
run_defense_summary = run_defense_summary[run_defense_summary['week'] >= cur_week_int-2]
defense_coverage_scheme = defense_coverage_scheme[defense_coverage_scheme['week'] >= cur_week_int-2]
defense_coverage_summary = defense_coverage_summary[defense_coverage_summary['week'] >= cur_week_int-2]
slot_coverage = slot_coverage[slot_coverage['week'] >= cur_week_int-2]

st_kickers = st_kickers[st_kickers['week'] >= cur_week_int-2]
st_punters = st_punters[st_punters['week'] >= cur_week_int-2]


In [ ]:
passing_depth = passing_depth.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
passing_allowed_pressure = passing_allowed_pressure.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
passing_pressure = passing_pressure.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
passing_concept = passing_concept.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
time_in_pocket = time_in_pocket.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
passing_summ_conc = passing_summ_conc.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)


rec_summ_conc = rec_summ_conc.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
receiving_concept =receiving_concept.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
receiving_depth = receiving_depth.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
receiving_scheme = receiving_scheme.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

rush_summ_conc =rush_summ_conc.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

block_summ_conc = block_summ_conc.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
offense_pass_blocking = offense_pass_blocking.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
offense_run_blocking = offense_run_blocking.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

def_summ_conc = def_summ_conc.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
pass_rush_summary = pass_rush_summary.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
run_defense_summary = run_defense_summary.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
defense_coverage_scheme = defense_coverage_scheme.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
defense_coverage_summary = defense_coverage_summary.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
slot_coverage = slot_coverage.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

st_kickers = st_kickers.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
st_punters = st_punters.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

In [ ]:
def df_nans(nData):
    a = np.full([nData.shape[0], nData.shape[1]], 0)
    np_df = pd.DataFrame(a, columns=nData.columns)
    ids = nData.iloc[:,:8].reset_index(drop=True)
    other_cols = np_df.iloc[:,8:]
    nData = pd.concat([ids, other_cols], axis=1)
    
    nData.replace(["NaN", 'NaT'], np.nan, inplace = True)
    nData=nData[pd.notnull(nData['player'])]
    #nData=nData.dropna(thresh=10)
    nData['year'] = cur_year_int
    nData['week'] = cur_week_int
    nData['plyr_number'] = cur_week_int
    nData['year'] = nData['year'].astype(int)
    nData['week'] = nData['week'].astype(int)
    return nData


passing_depth_new = df_nans(passing_depth_new)
passing_allowed_pressure_new = df_nans(passing_allowed_pressure_new)
passing_pressure_new = df_nans(passing_pressure_new)
passing_concept_new = df_nans(passing_concept_new)
time_in_pocket_new = df_nans(time_in_pocket_new)
passing_summ_conc_new = df_nans(passing_summ_conc_new)


rec_summ_conc = df_nans(rec_summ_conc)
receiving_concept =df_nans(receiving_concept)
receiving_depth = df_nans(receiving_depth)
receiving_scheme = df_nans(receiving_scheme)

rush_summ_conc = df_nans(rush_summ_conc)

block_summ_conc = df_nans(block_summ_conc)
offense_pass_blocking = df_nans(offense_pass_blocking)
offense_run_blocking = df_nans(offense_run_blocking)

def_summ_conc = df_nans(def_summ_conc)
pass_rush_summary = df_nans(pass_rush_summary)
run_defense_summary = df_nans(run_defense_summary)
defense_coverage_scheme = df_nans(defense_coverage_scheme)
defense_coverage_summary = df_nans(defense_coverage_summary)
slot_coverage = df_nans(slot_coverage)

st_kickers = df_nans(st_kickers)
st_punters = df_nans(st_punters)


In [ ]:
nfl_injuries = pd.read_csv('../nfl_models/current_data/week_'+cur_week+'/nfl_injuries.csv')
nfl_injuries['year'] = 2022

In [ ]:
def clean_name(df):
    result = ''.join(c for c in df['player'] if c.isalpha())
    return result.lower()
# file:///home/tomb/nfl_models/current_data/week_6/nfl_injuries.csv


nfl_injuries = pd.read_csv('../nfl_models/current_data/week_'+cur_week+'/nfl_injuries.csv')
nfl_injuries = nfl_injuries[['Player','Team','Pos','Injury','Status']]
nfl_injuries.columns = ['Player','Tm','Pos','Type','Class']
nfl_injuries.columns = [x.lower() for x in nfl_injuries.columns]
nfl_injuries = nfl_injuries[nfl_injuries['tm'] != 'FA']
nfl_injuries.drop_duplicates(subset=['player','tm','pos'], keep='first', inplace=True)
nfl_injuries['player'] = nfl_injuries.apply(lambda df: clean_name(df), axis=1)
nfl_injuries['year'] = 2022


def clean_non_player_dfs(df):
##  basic scrubbing to clean data ##
    df=df.apply(lambda x: x.astype(str).str.lower())
    df=df.replace('-','', regex=True)
    df['player']=df['player'].replace(' ','', regex=True)
    df['tm']=df['tm'].replace(' ','', regex=True)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['tm'].map(cleaning_dicts.clean_team_pfr).fillna(df['tm'])
    df['pos'] = df['pos'].map(cleaning_dicts.pos_dict).fillna(df['pos'])

    df.insert(0, "player_team", (df['player']+'_'+df['team_name']))
    ##add player_team_id## into the nfl_injuries cleaning function ###
    df.insert(1, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))


    return df

nfl_injuries = clean_non_player_dfs(nfl_injuries)
nfl_injuries['class']=nfl_injuries['class'].str.replace('pupr','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('nfir','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('nfia','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('inactive','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservecovid19','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservesus','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservednr','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reserveret','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservesus','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reserveex','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('pupp','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('irr','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('ir','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('doubtful','out')

nfl_injuries['class'].value_counts()

In [ ]:
pass_rush_summary

In [ ]:
passing_depth_new['player']= passing_depth_new.apply(lambda df: clean_name(df), axis=1)
passing_allowed_pressure_new['player']= passing_allowed_pressure_new.apply(lambda df: clean_name(df), axis=1)
passing_pressure_new['player']= passing_pressure_new.apply(lambda df: clean_name(df), axis=1)
passing_concept_new['player']= passing_concept_new.apply(lambda df: clean_name(df), axis=1)
time_in_pocket_new['player']= time_in_pocket_new.apply(lambda df: clean_name(df), axis=1)
passing_summ_conc_new['player']= passing_summ_conc_new.apply(lambda df: clean_name(df), axis=1)


rec_summ_conc['player']= rec_summ_conc.apply(lambda df: clean_name(df), axis=1)
receiving_concept['player']=receiving_concept.apply(lambda df: clean_name(df), axis=1)
receiving_depth['player']= receiving_depth.apply(lambda df: clean_name(df), axis=1)
receiving_scheme['player']= receiving_scheme.apply(lambda df: clean_name(df), axis=1)

rush_summ_conc['player']=rush_summ_conc.apply(lambda df: clean_name(df), axis=1)

block_summ_conc['player']= block_summ_conc.apply(lambda df: clean_name(df), axis=1)
offense_pass_blocking['player']= offense_pass_blocking.apply(lambda df: clean_name(df), axis=1)
offense_run_blocking['player']= offense_run_blocking.apply(lambda df: clean_name(df), axis=1)

def_summ_conc['player']= def_summ_conc.apply(lambda df: clean_name(df), axis=1)
pass_rush_summary['player']= pass_rush_summary.apply(lambda df: clean_name(df), axis=1)
run_defense_summary['player']= run_defense_summary.apply(lambda df: clean_name(df), axis=1)
defense_coverage_scheme['player']= defense_coverage_scheme.apply(lambda df: clean_name(df), axis=1)
defense_coverage_summary['player']= defense_coverage_summary.apply(lambda df: clean_name(df), axis=1)
slot_coverage['player']= slot_coverage.apply(lambda df: clean_name(df), axis=1)

st_kickers['player']= st_kickers.apply(lambda df: clean_name(df), axis=1)
st_punters['player']= st_punters.apply(lambda df: clean_name(df), axis=1)



In [ ]:
def new_ids(df):
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df


passing_depth_new = new_ids(passing_depth_new)
passing_allowed_pressure_new = new_ids(passing_allowed_pressure_new)
passing_pressure_new = new_ids(passing_pressure_new)
passing_concept_new = new_ids(passing_concept_new)
time_in_pocket_new = new_ids(time_in_pocket_new)
passing_summ_conc_new = new_ids(passing_summ_conc_new)


rec_summ_conc = new_ids(rec_summ_conc)
receiving_concept =new_ids(receiving_concept)
receiving_depth = new_ids(receiving_depth)
receiving_scheme = new_ids(receiving_scheme)

rush_summ_conc = new_ids(rush_summ_conc)

block_summ_conc = new_ids(block_summ_conc)
offense_pass_blocking = new_ids(offense_pass_blocking)
offense_run_blocking = new_ids(offense_run_blocking)

def_summ_conc = new_ids(def_summ_conc)
pass_rush_summary = new_ids(pass_rush_summary)
run_defense_summary = new_ids(run_defense_summary)
defense_coverage_scheme = new_ids(defense_coverage_scheme)
defense_coverage_summary = new_ids(defense_coverage_summary)
slot_coverage = new_ids(slot_coverage)

st_kickers = new_ids(st_kickers)
st_punters_no_inj = new_ids(st_punters)

In [ ]:
from fuzzywuzzy import fuzz

def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = 0
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

def fuzzy_inj_match(df):
    # List for dicts for easy dataframe creation
    dict_list = []
    # iterating over our players without salaries found above
    for name in df.player_team_id:
        # Use our method to find best match, we can set a threshold here
        match = match_name(name, nfl_injuries.player_team_id, 85)
        if match[1] == 0:
            pass
        else:
            # New dict for storing data
            dict_ = {}
            dict_.update({"player_team_id" : name})
            dict_.update({"match_name" : match[0]})
            dict_.update({"score" : match[1]})
            dict_list.append(dict_)
    return pd.DataFrame(dict_list)
        #return pd.merge(df, merge_table, left_on='unique_player_id', right_on='player_name', how='left')

# Get matching names from injury report to pff player pools
passing_depth_new_match = fuzzy_inj_match(passing_depth_new)
passing_allowed_pressure_new_match = fuzzy_inj_match(passing_allowed_pressure_new)
passing_pressure_new_match = fuzzy_inj_match(passing_pressure_new)
passing_concept_new_match = fuzzy_inj_match(passing_concept_new)
time_in_pocket_new_match = fuzzy_inj_match(time_in_pocket_new)
passing_summ_conc_new_match= fuzzy_inj_match(passing_summ_conc_new)


rec_summ_conc_match = fuzzy_inj_match(rec_summ_conc)
receiving_concept_match =fuzzy_inj_match(receiving_concept)
receiving_depth_match = fuzzy_inj_match(receiving_depth)
receiving_scheme_match = fuzzy_inj_match(receiving_scheme)

rush_summ_conc_match = fuzzy_inj_match(rush_summ_conc)

block_summ_conc_match = fuzzy_inj_match(block_summ_conc)
offense_pass_blocking_match = fuzzy_inj_match(offense_pass_blocking)
offense_run_blocking_match = fuzzy_inj_match(offense_run_blocking)

def_summ_conc_match = fuzzy_inj_match(def_summ_conc)
pass_rush_summary_match = fuzzy_inj_match(pass_rush_summary)
run_defense_summary_match = fuzzy_inj_match(run_defense_summary)
defense_coverage_scheme_match = fuzzy_inj_match(defense_coverage_scheme)
defense_coverage_summary_match = fuzzy_inj_match(defense_coverage_summary)
slot_coverage_match = fuzzy_inj_match(slot_coverage)

st_kickers_match = fuzzy_inj_match(st_kickers)
st_punters_match = fuzzy_inj_match(st_punters)

In [ ]:
## Subset nfl_injuries down to  ##
nfl_injuries = nfl_injuries[['player_team_id','pos','type','class']]
nfl_injuries['class']=np.where(nfl_injuries['class'] == "questionable", 1, 0)

## merge it onto our dataframes and where the value is zero, we need to remove them ##
passing_depth_new_match = pd.merge(passing_depth_new_match, nfl_injuries, on='player_team_id', how='left')
passing_allowed_pressure_new_match = pd.merge(passing_allowed_pressure_new_match, nfl_injuries, on='player_team_id', how='left')
passing_pressure_new_match = pd.merge(passing_pressure_new_match, nfl_injuries, on='player_team_id', how='left')
passing_concept_new_match = pd.merge(passing_concept_new_match, nfl_injuries, on='player_team_id', how='left')
time_in_pocket_new_match = pd.merge(time_in_pocket_new_match, nfl_injuries, on='player_team_id', how='left')
passing_summ_conc_new_match = pd.merge(passing_summ_conc_new_match, nfl_injuries, on='player_team_id', how='left')


rec_summ_conc_match = pd.merge(rec_summ_conc_match, nfl_injuries, on='player_team_id', how='left')
receiving_concept_match =pd.merge(receiving_concept_match, nfl_injuries, on='player_team_id', how='left')
receiving_depth_match = pd.merge(receiving_depth_match, nfl_injuries, on='player_team_id', how='left')
receiving_scheme_match = pd.merge(receiving_scheme_match, nfl_injuries, on='player_team_id', how='left')

rush_summ_conc_match =pd.merge(rush_summ_conc_match, nfl_injuries, on='player_team_id', how='left')

block_summ_conc_match = pd.merge(block_summ_conc_match, nfl_injuries, on='player_team_id', how='left')
offense_pass_blocking_match = pd.merge(offense_pass_blocking_match, nfl_injuries, on='player_team_id', how='left')
offense_run_blocking_match = pd.merge(offense_run_blocking_match, nfl_injuries, on='player_team_id', how='left')

def_summ_conc_match = pd.merge(def_summ_conc_match, nfl_injuries, on='player_team_id', how='left')
pass_rush_summary_match = pd.merge(pass_rush_summary_match, nfl_injuries, on='player_team_id', how='left')
run_defense_summary_match = pd.merge(run_defense_summary_match, nfl_injuries, on='player_team_id', how='left')
defense_coverage_scheme_match = pd.merge(defense_coverage_scheme_match, nfl_injuries, on='player_team_id', how='left')
defense_coverage_summary_match = pd.merge(defense_coverage_summary_match, nfl_injuries, on='player_team_id', how='left')
slot_coverage_match = pd.merge(slot_coverage_match, nfl_injuries, on='player_team_id', how='left')

st_kickers_match = pd.merge(st_kickers_match, nfl_injuries, on='player_team_id', how='left')
#st_punters_match = pd.merge(st_punters_match, nfl_injuries, on='player_team_id', how='left')
rush_summ_conc_match['class'].value_counts()

In [ ]:
rush_summ_conc_match.head()

In [ ]:
passing_depth_out = passing_depth_new_match.loc[(passing_depth_new_match['class'] == 0)]
passing_allowed_pressure_out= passing_allowed_pressure_new_match.loc[(passing_allowed_pressure_new_match['class'] == 0)]
passing_pressure_out = passing_pressure_new_match.loc[(passing_pressure_new_match['class'] == 0)]
passing_concept_out = passing_concept_new_match.loc[(passing_concept_new_match['class'] == 0)]
time_in_pocket_out = time_in_pocket_new_match.loc[(time_in_pocket_new_match['class'] == 0)]
passing_summ_conc_out= passing_summ_conc_new_match.loc[(passing_summ_conc_new_match['class'] == 0)]


rec_summ_conc_out = rec_summ_conc_match.loc[(rec_summ_conc_match['class'] == 0)]
receiving_concept_out =receiving_concept_match.loc[(receiving_concept_match['class'] == 0)]
receiving_depth_out = receiving_depth_match.loc[(receiving_depth_match['class'] == 0)]
receiving_scheme_out = receiving_scheme_match.loc[(receiving_scheme_match['class'] == 0)]

rush_summ_conc_out = rush_summ_conc_match.loc[(rush_summ_conc_match['class'] == 0)]

block_summ_conc_out = block_summ_conc_match.loc[(block_summ_conc_match['class'] == 0)]
offense_pass_blocking_out = offense_pass_blocking_match.loc[(offense_pass_blocking_match['class'] == 0)]
offense_run_blocking_out = offense_run_blocking_match.loc[(offense_run_blocking_match['class'] == 0)]

def_summ_conc_out = def_summ_conc_match.loc[(def_summ_conc_match['class'] == 0)]
pass_rush_summary_out = pass_rush_summary_match.loc[(pass_rush_summary_match['class'] == 0)]
run_defense_summary_out = run_defense_summary_match.loc[(run_defense_summary_match['class'] == 0)]
defense_coverage_scheme_out = defense_coverage_scheme_match.loc[(defense_coverage_scheme_match['class'] == 0)]
defense_coverage_summary_out = defense_coverage_summary_match.loc[(defense_coverage_summary_match['class'] == 0)]
slot_coverage_out = slot_coverage_match.loc[(slot_coverage_match['class'] == 0)]

st_kickers_out = st_kickers_match.loc[(st_kickers_match['class'] == 0)]
#st_punters_out = st_punters_match.loc[(st_punters_match['class'] == 0)]



passing_depth_quest = passing_depth_new_match.loc[(passing_depth_new_match['class'] == 1)]
passing_allowed_pressure_quest= passing_allowed_pressure_new_match.loc[(passing_allowed_pressure_new_match['class'] == 1)]
passing_pressure_quest = passing_pressure_new_match.loc[(passing_pressure_new_match['class'] == 1)]
passing_concept_quest = passing_concept_new_match.loc[(passing_concept_new_match['class'] == 1)]
time_in_pocket_quest = time_in_pocket_new_match.loc[(time_in_pocket_new_match['class'] == 1)]
passing_summ_conc_quest= passing_summ_conc_new_match.loc[(passing_summ_conc_new_match['class'] == 1)]


rec_summ_conc_quest = rec_summ_conc_match.loc[(rec_summ_conc_match['class'] == 1)]
receiving_concept_quest =receiving_concept_match.loc[(receiving_concept_match['class'] == 1)]
receiving_depth_quest = receiving_depth_match.loc[(receiving_depth_match['class'] == 1)]
receiving_scheme_quest = receiving_scheme_match.loc[(receiving_scheme_match['class'] == 1)]

rush_summ_conc_quest = rush_summ_conc_match.loc[(rush_summ_conc_match['class'] == 1)]

block_summ_conc_quest = block_summ_conc_match.loc[(block_summ_conc_match['class'] == 1)]
offense_pass_blocking_quest = offense_pass_blocking_match.loc[(offense_pass_blocking_match['class'] == 1)]
offense_run_blocking_quest = offense_run_blocking_match.loc[(offense_run_blocking_match['class'] == 1)]

def_summ_conc_quest = def_summ_conc_match.loc[(def_summ_conc_match['class'] == 1)]
pass_rush_summary_quest = pass_rush_summary_match.loc[(pass_rush_summary_match['class'] == 1)]
run_defense_summary_quest = run_defense_summary_match.loc[(run_defense_summary_match['class'] == 1)]
defense_coverage_scheme_quest = defense_coverage_scheme_match.loc[(defense_coverage_scheme_match['class'] == 1)]
defense_coverage_summary_quest = defense_coverage_summary_match.loc[(defense_coverage_summary_match['class'] == 1)]
slot_coverage_quest = slot_coverage_match.loc[(slot_coverage_match['class'] == 1)]

st_kickers_quest = st_kickers_match.loc[(st_kickers_match['class'] == 1)]
#st_punters_quest = st_punters_match.loc[(st_punters_match['class'] == 1)]

In [ ]:
injured_players_list= pd.concat([
passing_depth_out, 
passing_allowed_pressure_out, 
passing_pressure_out, 
passing_concept_out, 
time_in_pocket_out, 
passing_summ_conc_out, 
rec_summ_conc_out, 
receiving_concept_out,
receiving_depth_out, 
receiving_scheme_out, 
rush_summ_conc_out,
block_summ_conc_out, 
offense_pass_blocking_out, 
offense_run_blocking_out, 
def_summ_conc_out, 
pass_rush_summary_out, 
run_defense_summary_out, 
defense_coverage_scheme_out, 
defense_coverage_summary_out, 
slot_coverage_out, 
st_kickers_out], axis=0)

injured_players_list.drop_duplicates(inplace=True)

In [ ]:
passing_depth_new = passing_depth_new[~passing_depth_new['player_team_id'].isin(injured_players_list['match_name'])]
passing_allowed_pressure_new = passing_allowed_pressure_new[~passing_allowed_pressure_new['player_team_id'].isin(injured_players_list['match_name'])]
passing_pressure_new = passing_pressure_new[~passing_pressure_new['player_team_id'].isin(injured_players_list['match_name'])]
passing_concept_new = passing_concept_new[~passing_concept_new['player_team_id'].isin(injured_players_list['match_name'])]
time_in_pocket_new = time_in_pocket_new[~time_in_pocket_new['player_team_id'].isin(injured_players_list['match_name'])]
passing_summ_conc_new= passing_summ_conc_new[~passing_summ_conc_new['player_team_id'].isin(injured_players_list['match_name'])]


rec_summ_conc = rec_summ_conc[~rec_summ_conc['player_team_id'].isin(injured_players_list['match_name'])]
receiving_concept =receiving_concept[~receiving_concept['player_team_id'].isin(injured_players_list['match_name'])]
receiving_depth = receiving_depth[~receiving_depth['player_team_id'].isin(injured_players_list['match_name'])]
receiving_scheme = receiving_scheme[~receiving_scheme['player_team_id'].isin(injured_players_list['match_name'])]

rush_summ_conc = rush_summ_conc[~rush_summ_conc['player_team_id'].isin(injured_players_list['match_name'])]

block_summ_conc = block_summ_conc[~block_summ_conc['player_team_id'].isin(injured_players_list['match_name'])]
offense_pass_blocking = offense_pass_blocking[~offense_pass_blocking['player_team_id'].isin(injured_players_list['match_name'])]
offense_run_blocking = offense_run_blocking[~offense_run_blocking['player_team_id'].isin(injured_players_list['match_name'])]

def_summ_conc = def_summ_conc[~def_summ_conc['player_team_id'].isin(injured_players_list['match_name'])]
pass_rush_summary = pass_rush_summary[~pass_rush_summary['player_team_id'].isin(injured_players_list['match_name'])]
run_defense_summary = run_defense_summary[~run_defense_summary['player_team_id'].isin(injured_players_list['match_name'])]
defense_coverage_scheme = defense_coverage_scheme[~defense_coverage_scheme['player_team_id'].isin(injured_players_list['match_name'])]
defense_coverage_summary = defense_coverage_summary[~defense_coverage_summary['player_team_id'].isin(injured_players_list['match_name'])]
slot_coverage = slot_coverage[~slot_coverage['player_team_id'].isin(injured_players_list['match_name'])]

st_kickers = st_kickers[~st_kickers['player_team_id'].isin(injured_players_list['match_name'])]
#st_punters = st_punters[~st_punters['player_team_id'].isin(injured_players_list['match_name'])]

In [ ]:
slot_coverage

In [ ]:
passing_depth_new.to_csv('./current_data/week_'+cur_week_str+'/passing_depth_new_pp_w'+cur_week+'.csv', index=False)
passing_allowed_pressure_new.to_csv('./current_data/week_'+cur_week_str+'/passing_allowed_pressure_new_pp_w'+cur_week+'.csv', index=False)
passing_pressure_new.to_csv('./current_data/week_'+cur_week_str+'/passing_pressure_new_pp_w'+cur_week+'.csv', index=False)
passing_concept_new.to_csv('./current_data/week_'+cur_week_str+'/passing_concept_new_pp_w'+cur_week+'.csv', index=False)
time_in_pocket_new.to_csv('./current_data/week_'+cur_week_str+'/time_in_pocket_new_pp_w'+cur_week+'.csv', index=False)
passing_summ_conc_new.to_csv('./current_data/week_'+cur_week_str+'/passing_summ_conc_new_pp_w'+cur_week+'.csv', index=False)


rec_summ_conc.to_csv('./current_data/week_'+cur_week_str+'/rec_summ_conc_pp_w'+cur_week+'.csv', index=False)
receiving_concept.to_csv('./current_data/week_'+cur_week_str+'/receiving_concept_pp_w'+cur_week+'.csv', index=False)
receiving_depth.to_csv('./current_data/week_'+cur_week_str+'/receiving_depth_pp_w'+cur_week+'.csv', index=False)
receiving_scheme.to_csv('./current_data/week_'+cur_week_str+'/receiving_scheme_pp_w'+cur_week+'.csv', index=False)

rush_summ_conc.to_csv('./current_data/week_'+cur_week_str+'/rush_summ_conc_pp_w'+cur_week+'.csv', index=False)

block_summ_conc.to_csv('./current_data/week_'+cur_week_str+'/block_summ_conc_pp_w'+cur_week+'.csv', index=False)
offense_pass_blocking.to_csv('./current_data/week_'+cur_week_str+'/offense_pass_blocking_pp_w'+cur_week+'.csv', index=False)
offense_run_blocking.to_csv('./current_data/week_'+cur_week_str+'/offense_run_blocking_pp_w'+cur_week+'.csv', index=False)

def_summ_conc.to_csv('./current_data/week_'+cur_week_str+'/def_summ_conc_pp_w'+cur_week+'.csv', index=False)
pass_rush_summary.to_csv('./current_data/week_'+cur_week_str+'/pass_rush_summary_pp_w'+cur_week+'.csv', index=False)
run_defense_summary.to_csv('./current_data/week_'+cur_week_str+'/run_defense_summary_pp_w'+cur_week+'.csv', index=False)
defense_coverage_scheme.to_csv('./current_data/week_'+cur_week_str+'/defense_coverage_scheme_pp_w'+cur_week+'.csv', index=False)
defense_coverage_summary.to_csv('./current_data/week_'+cur_week_str+'/defense_coverage_summary_pp_w'+cur_week+'.csv', index=False)
slot_coverage.to_csv('./current_data/week_'+cur_week_str+'/slot_coverage_pp_w'+cur_week+'.csv', index=False)

st_kickers.to_csv('./current_data/week_'+cur_week_str+'/st_kickers_pp_w'+cur_week+'.csv', index=False)
st_punters_no_inj.to_csv('./current_data/week_'+cur_week_str+'/st_punters_no_inj_pp_w'+cur_week+'.csv', index=False)